# Recipes

This page references recipes for common operations for the PyHMMER API, similarly to the `itertools` [recipes](https://docs.python.org/3/library/itertools.html#itertools-recipes) from the Python documentation.

## Loading multiple HMMs

An adapater for wrapping several `HMMFile` objects into a single one.

<div class="alert alert-info">
    
Credits
    
Original implementation proposed by [Zachary Kurtz](https://github.com/zdk123) in [#24](https://github.com/althonos/pyhmmer/issues/24), updated to use an `ExitStack` for managing the files.
    
</div>

In [ ]:
import contextlib
import itertools
import os
import typing

from pyhmmer.plan7 import HMMFile, HMM

class HMMFiles(typing.ContextManager[typing.Iterable[HMM]]):
    def __init__(self, *files: os.PathLike[bytes]) -> None:
        self.stack = contextlib.ExitStack()
        self.hmmfiles = [self.stack.enter_context(HMMFile(f)) for f in files]

    def __enter__(self) -> typing.Iterable[HMM]:
        return itertools.chain.from_iterable(self.hmmfiles)

    def __exit__(self, exc_value: object, exc_type: object, traceback: object) -> None:
        self.stack.close()

To use it with `hmmsearch`, simply create a `HMMFiles` object with the paths for the different HMM files to concatenate. They will be read in the order given as argument.

In [ ]:
import pyhmmer
from pyhmmer.easel import SequenceFile

with SequenceFile("data/seqs/938293.PRJEB85.HG003687.faa", digital=True) as sequences:
    targets = sequences.read_block()
    
with HMMFiles("data/hmms/txt/PKSI-AT.hmm", "data/hmms/txt/LuxC.hmm") as hmm_files:
    all_hits = list(pyhmmer.hmmsearch(hmm_files, targets))
    
print("HMMs searched:", len(all_hits))
print("Hits found:   ", sum(len(hits) for hits in all_hits))